In [1]:
pwd

'C:\\Users\\hamza\\Programs\\Road_Traffic_Monitoring_AI_Vision_Based_SMART_sensor\\notebooks'

In [2]:
cd ..

C:\Users\hamza\Programs\Road_Traffic_Monitoring_AI_Vision_Based_SMART_sensor


C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import time

#####################################################################################################################################

import cv2
import numpy as np
import torch

#####################################################################################################################################

from framegrabber.frame_grabber import FrameGrabber

from detection.ultralytics_detectors import UltralyticsDetector

from tracking.track import Tracker

from counting.count_config_loader import CountingConfigLoader
from counting.count_visualizer import CountVisualizer

#####################################################################################################################################

my_model = UltralyticsDetector("yolo11n_finetuned.pt" , conf=0.50)         # rtdetr-l.pt  yolo11n.pt yolo26n.pt yolo11n_finetuned
coco_vehicles = [1, 2, 3, 5, 7]
vehicles_4_wheels = [2, 5, 7]
device = my_model.predictor.device

Ultralytics 8.4.5  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs
Successfully yolo11n_finetuned.pt model is initialized and warmedup !


2026-02-12 19:16:03.453 | MainProcess/MainThread | INFO     | C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\boxmot\trackers\basetracker.py:56 | __init__ - BaseTracker initialization parameters:
2026-02-12 19:16:03.455 | MainProcess/MainThread | INFO     | C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\boxmot\trackers\basetracker.py:57 | __init__ - det_thresh: 0.3
2026-02-12 19:16:03.456 | MainProcess/MainThread | INFO     | C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\boxmot\trackers\basetracker.py:58 | __init__ - max_age: 30
2026-02-12 19:16:03.456 | MainProcess/MainThread | INFO     | C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\boxmot\trackers\basetracker.py:59 | __init__ - max_obs: 50
2026-02-12 19:16:03.456 | MainProcess/MainThread | INFO     | C:\Users\hamza\anaconda3\envs\smart_sensor\lib\site-packages\boxmot\trackers\basetracker.py:60 | __init__ - min_hits: 3
2026-02-12 19:16:03.456 | MainProcess/MainThread | INFO    

In [4]:
my_model.predictor.args.conf , my_model.predictor.args.imgsz

(0.5, [640])

In [5]:
from detection.torchvision_detectors import TorchvisionDetector

det = TorchvisionDetector(
    "fasterrcnn_resnet50_fpn_v2",
    conf=0.6,
    device="cuda"
)

# detections = det.detect_to_track(frame.data)

* yolo26n: YOLO26n summary (fused): 122 layers, 2,408,932 parameters, 0 gradients, 5.4 GFLOPs
* YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
* YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

In [6]:
def ordered_counters(*counters):
    """
    Accepts:
      ([roi1, roi2], global_ctr)
      or
      ([roi1, roi2, global_ctr],)

    Returns:
      flat, ordered list of counter objects
    """

    # ---- flatten ----
    flat = []
    for c in counters:
        if isinstance(c, (list, tuple)):
            flat.extend(c)
        else:
            flat.append(c)

    # ---- semantic order ----
    def key(ctr):
        info = ctr.get_area_info()
        # ROI / line first, global last
        if info.get("polygon") is not None or info.get("line") is not None:
            return (0, info.get("name", ""))
        return (1, info.get("name", ""))

    return sorted(flat, key=key)

    
count_vis = CountVisualizer(
    show_legend=False,   # hide class legend
    show_summary=True    # keep total summary box
)

counter_load = CountingConfigLoader()

counters = counter_load.load_counting_areas()

counters = ordered_counters(*counters)
counters

In [7]:
from utils.profilers import Profile

device = my_model.predictor.device
inf_profile = Profile(device=device)
pre_profile = Profile(device=device)
post_profile = Profile(device=device)
track_profile = Profile()
count_profile = Profile()
grabber_profile = Profile()
plot_profile = Profile()

In [8]:
tracking_method = "bytetrack"
tracker = Tracker(tracking_method)

In [9]:
# source = r"C:\Users\hamza\Datasets\TrafficDatasets\IMAROC_2\kech66.mp4"                   # 0 "kech.mp4" , "vid1.mp4"
# stride = 3
# stride_method = "periodic_stride"             # "burst_stride", "periodic_stride", "random_sampling"

# frame_grabber = FrameGrabber(source, stride=stride, stride_method=stride_method)


# if not frame_grabber.open():
#     raise RuntimeError("Failed to open source")
# # ensure window exists (main thread)
# if frame_grabber._grabber_mode=="queue":
# # start producer
#     frame_grabber.start()

# try:
#     with torch.inference_mode():
#         while True:
#             with grabber_profile:
#                 # try to get a frame but don't block forever
#                 frame = frame_grabber.get_frame(timeout=0.1)  # <-- short timeout keeps loop responsive
                    
#             if frame is not None:
#                 print(f"frame_grabber index: {frame.read_idx}")

#                 with inf_profile:
#                     ready_to_track_array = my_model.detect_to_track(frame.data)
                    
#                 with track_profile:
#                     res = tracker.update(ready_to_track_array , frame.data)
                    
#                 det_array_plot = my_model.plot()


#                 with count_profile:
#                     g_count = counters[0].count(res)
                    
#                 # mark processed & show
#                 frame_grabber.mark_processed(frame)

                
#             else:
#                 # no frame this iteration (timeout), you may choose to display a placeholder
#                 # or simply continue — but still poll for key events below
#                 break

# finally:
#     frame_grabber.release()

In [10]:
# g_count.total_count

In [25]:
source = r"C:\Users\hamza\Datasets\TrafficDatasets\IMAROC_2\kech37.mp4"                   # 0 "kech.mp4" , "vid1.mp4"

# --- Video writer setup ---
output_path = "output_counting_37_.mp4"

fps = 30

# Get frame size (wait until first frame if needed)
ret, test_cap = cv2.VideoCapture(source).read()
h, w = test_cap.shape[:2]

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # widely supported
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

assert video_writer.isOpened(), "Failed to open VideoWriter"









stride = 2
stride_method = "periodic_stride"             # "burst_stride", "periodic_stride", "random_sampling"

frame_grabber = FrameGrabber(source, stride=stride, stride_method=stride_method)


if not frame_grabber.open():
    raise RuntimeError("Failed to open source")
# ensure window exists (main thread)
cv2.namedWindow('BoXMOT + ultralytics', cv2.WINDOW_NORMAL)
if frame_grabber._grabber_mode=="queue":
# start producer
    frame_grabber.start()

try:
    with torch.inference_mode():
        while True:
            with grabber_profile:
                # try to get a frame but don't block forever
                frame = frame_grabber.get_frame(timeout=0.1)  # <-- short timeout keeps loop responsive
                    
            if frame is not None:
                print(f"frame_grabber index: {frame.read_idx}")

                with inf_profile:
                    ready_to_track_array = my_model.detect_to_track(frame.data)
   
                with track_profile:
                    res = tracker.update(ready_to_track_array , frame.data)
                    # print(f"tracking array: {res} for counter: {counters[2]}")
                # det_array_plot = my_model.plot()
                # det_array_plot = det.plot(frame.data, detections)
                
                track_array_plot = tracker.tracker.plot_results(frame.data, show_trajectories=True)

                with count_profile:
                    g_count = counters[2].count(res)
                    roi1_count = counters[0].count(res)
                    roi2_count = counters[1].count(res)
                    
                with plot_profile:
                    count_plot = count_vis.render(track_array_plot, *counters)


                # --- WRITE FRAME TO VIDEO ---
                video_writer.write(count_plot)





                
                # Speed
                est.update(res, frame_idx=frame.read_idx, timestamp=None)
                # speeds = est.get_and_clear_speeds()
                if speeds:
                    print(f"Frame {frame}: computed speeds -> {speeds}")


                    
                    
                # mark processed & show
                frame_grabber.mark_processed(frame)

                cv2.imshow('BoXMOT + ultralytics', count_plot)
                
            else:
                # no frame this iteration (timeout), you may choose to display a placeholder
                # or simply continue — but still poll for key events below
                pass

            # ALWAYS poll keyboard events so 'q' is detected even when no frame was available
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                # stop producer and break loop
                if frame_grabber._grabber_mode=="queue":
                    frame_grabber.stop(wait=True)
                break
    
                # optional: also break when producer finished (sentinel)
                if frame_grabber._grabber_mode=="queue":
                    if frame is None and frame_grabber._stop_event.is_set():
                        break
finally:
    frame_grabber.release()
    video_writer.release()   # <-- ADD THIS
    cv2.destroyAllWindows()

frame_grabber index: 0
frame_grabber index: 2
frame_grabber index: 4
frame_grabber index: 6
frame_grabber index: 8
frame_grabber index: 10
frame_grabber index: 12
frame_grabber index: 14
frame_grabber index: 16
frame_grabber index: 18
frame_grabber index: 20
frame_grabber index: 22
frame_grabber index: 24
frame_grabber index: 26
frame_grabber index: 28
frame_grabber index: 30
frame_grabber index: 32
frame_grabber index: 34
frame_grabber index: 36
frame_grabber index: 38
frame_grabber index: 40
frame_grabber index: 42
frame_grabber index: 44
frame_grabber index: 46
frame_grabber index: 48
frame_grabber index: 50
frame_grabber index: 52
frame_grabber index: 54
frame_grabber index: 56
frame_grabber index: 58
frame_grabber index: 60
frame_grabber index: 62
frame_grabber index: 64
frame_grabber index: 66
frame_grabber index: 68
frame_grabber index: 70
frame_grabber index: 72
frame_grabber index: 74
frame_grabber index: 76
frame_grabber index: 78
frame_grabber index: 80
frame_grabber index: 

In [ ]:
g_count , roi1_count , roi2_count

In [ ]:
g_count.total_count , g_count.counts_by_class

In [14]:
from utils.shape_setter import PointSelector , LineSelector , TwoLineSelector , PolygonSelector , RectangleSelector , OBBSelector
import cv2
import numpy as np
source = r"C:\Users\hamza\Datasets\TrafficDatasets\IMAROC_2\kech37.mp4" 

stride = 1
stride_method = "periodic_stride"             # "burst_stride", "periodic_stride", "random_sampling"

cap = cv2.VideoCapture(source)
ok, first_frame = cap.read()
cap.release()

if not ok or first_frame is None:
    print("Failed to read example frame; please provide a valid path ('kech.mp4' used in example).")
else:
    # # line
    # line_sel = LineSelector(max_display_size=900, auto_confirm=True, preview_wait_secs=None)
    # line = line_sel.select_line(first_frame)
    # print("Selected line:", line)

    # # two lines
    # two_sel = TwoLineSelector(max_display_size=900, auto_confirm=True, preview_wait_secs=None)
    # two = two_sel.select_two_lines(first_frame)
    # print("Selected two lines:", two)

    #polygon
    poly_sel = PolygonSelector(max_display_size=900, min_points=4, auto_close_on_click_near_first=True,
                               close_pixel_radius=12, preview_wait_secs=None)
    poly = poly_sel.select_polygon(first_frame)
    print("Selected polygon:", poly)

    # # rectangle
    # rect_sel = RectangleSelector(max_display_size=900, auto_confirm=True, preview_wait_secs=None)
    # rect = rect_sel.select_rectangle(first_frame)
    # if rect is None:
    #     print("Cancelled")
    # else:
    #     print("Selected rectangle:", rect)


    # # # obb£
    # obb_selector = OBBSelector()
    # obb_points = obb_selector.select_obb(first_frame)
    # if obb_points is not None:
    #     print("Selected OBB points:", obb_points)
    # else:
    #     print("Selection cancelled")


    # selector = PointSelector()
    # pt = selector.select_point(first_frame)
    # print("Selected point:", pt)


Selected polygon: None


* I need the sys metrics file
* I need the actual counts
* I need to understand why counting fails ?????? or it takes a very long time to operate !!!!!